### Preliminaries

In [115]:
import praw
import pandas as pd
from get_all_tickers import get_tickers as gt
from math import log
from datetime import datetime
from datetime import timedelta
from yahoo_earnings_calendar import YahooEarningsCalendar
import dateutil.parser
import yfinance as yf
from pandas.tseries.offsets import CustomBusinessDay
import matplotlib.pyplot as plt
import plotly.express as px
import heapq
from scipy import stats

In [ ]:
### Credentials - KEEP SECRET!
client_id, client_secret, user_agent = ('XXXXXXXXXXXXXXXX', 'XXXXXXXXXXXXXXXX', 'XXXXXXXXXXXXXXXX')

In [ ]:
# Access Reddit and WSB
reddit = praw.Reddit(client_id=client_id, client_secret=client_secret, user_agent=user_agent)
wsb = reddit.subreddit("wallstreetbets")

### Collecting Comments

In [ ]:
# get earnings threads
earnings_thread_ids = ['iiqpiz', 'imzuci', 'i1rkto', 'ia6lcr', 'i5xmeg', 
                    'iefzz8', 'hpah3l', 'hl8ug8', 'htg9xp', 'hxmlgh']

comments = []

for thread_id in earnings_thread_ids:
    thread = reddit.submission(id=thread_id)
    
    thread.comments.replace_more(limit=None) # same as the "load more comments" feature on Reddit
    for top_level_comment in thread.comments:
        comments.append((top_level_comment.body, top_level_comment.score))

In [ ]:
# convert to pandas
comments = pd.DataFrame(comments, columns=['text', 'score'])

In [ ]:
# save to csv
comments.to_csv('comments.csv', index=False)

### Processing Comments

Manually add tickers and sentiment...

Read it back in and clean it up a bit more

In [7]:
# read from csv (now with tickers and sentiment)
cleaned_comments = pd.read_csv('comments_manual.csv')

In [8]:
def ticker_list(tickers):
    lst = tickers.split(', ')
    return lst

In [9]:
cleaned_comments['tickers'] = cleaned_comments['tickers'].apply(ticker_list)

In [10]:
# log_upvotes is log of upvotes; sentimement_score also indicates direction (negative for bearish, positive for bullish)

cleaned_comments['log_upvotes'] = cleaned_comments['score'].apply(lambda x: log(x+1))
cleaned_comments['sentiment_score'] = cleaned_comments['log_upvotes']*cleaned_comments['sentiment']

In [11]:
cleaned_comments

,text,score,tickers,sentiment,log_upvotes,sentiment_score
0,Smith & Wesson Calls 🤔 gun sales must be high ...,28,[SWBI],1,3.367296,3.367296
1,Docusign might be worthy,8,[DOCU],1,2.197225,2.197225
2,Micheals might be amazing tbh,8,[MIK],1,2.197225,2.197225
3,Anyone else in JAMF calls leading up to earnin...,8,[JAMF],1,2.197225,2.197225
4,I imagine jamf could be great with companys al...,4,[JAMF],1,1.609438,1.609438
...,...,...,...,...,...,...
400,"Aphria, McDonald's and Amazon.",1,"[APHA, MCD, AMZN]",1,0.693147,0.693147
401,UPS will absolutely destroy earnings.,1,[UPS],1,0.693147,0.693147
402,Im so excited about APHA loaded a lot of stock...,1,[APHA],1,0.693147,0.693147
403,APPL open green tomorrow?,1,[AAPL],1,0.693147,0.693147


In [12]:
# get a set of all the tickers that have been mentioned
all_tickers = set()

for l in cleaned_comments['tickers']:
    for t in l:
        all_tickers.add(t)
        
all_tickers = list(all_tickers)
len(all_tickers)

178

but really, we want to compare this with earnings.

### Collecting Earnings data

In [ ]:
q2_earnings_calendar = pd.DataFrame()
start_date = datetime(2020, 7, 5)
end_date = datetime(2020, 9, 12)
yec = YahooEarningsCalendar(0)
progress = 0

for ticker in all_tickers:
    progress +=1
    print(str(progress) + '/' + str(len(all_tickers)) + ': ' + ticker)
    hist_earn_dates = pd.DataFrame(yec.get_earnings_of(ticker))
    if not(hist_earn_dates.empty):
        hist_earn_dates['startdatetime'] = hist_earn_dates['startdatetime'].apply(lambda x:datetime.fromisoformat(x[:-1]))
        hist_earn_dates = hist_earn_dates.loc[(hist_earn_dates['startdatetime'] > start_date) &
                                              (hist_earn_dates['startdatetime'] < end_date)]
        q2_earnings_calendar = q2_earnings_calendar.append(hist_earn_dates)

In [ ]:
# save dataframe (because downloading takes a long time)
q2_earnings_calendar.to_csv('earnings_calendar.csv', index=False)

A few duplicates and manual fixes... but ok

In [13]:
# read back in
q2_earnings_calendar = pd.read_csv('earnings_calendar_fixed.csv')

In [14]:
q2_earnings_calendar['startdatetime'] = q2_earnings_calendar['startdatetime'].apply(lambda x:datetime.fromisoformat(x[:10]))

In [15]:
q2_earnings_calendar

,ticker,companyshortname,startdatetime,startdatetimetype,epsestimate,epsactual,epssurprisepct,timeZoneShortName,gmtOffsetMilliSeconds,quoteType
0,HRB,"H&R Block, Inc.",2020-09-01,TNS,0.50,0.55,10.44,EDT,0,EQUITY
1,CLDR,"Cloudera, Inc.",2020-09-02,TNS,0.07,0.10,53.85,EDT,0,EQUITY
2,AAPL,Apple Inc.,2020-07-30,TNS,0.51,0.65,26.22,EDT,0,EQUITY
3,AXP,American Express Company,2020-07-24,TNS,-0.11,0.29,354.39,EDT,0,EQUITY
4,ROST,"Ross Stores, Inc.",2020-08-20,TNS,-0.27,0.06,122.56,EDT,0,EQUITY
...,...,...,...,...,...,...,...,...,...,...
173,MRO,Marathon Oil Corporation,2020-08-05,TNS,-0.63,-0.60,5.21,EDT,0,EQUITY
174,GME,GameStop Corp.,2020-09-09,TNS,-1.13,-1.40,-23.35,EDT,0,EQUITY
175,DELL,Dell Technologies Inc.,2020-08-27,TNS,1.40,1.92,37.63,EDT,0,EQUITY
176,SNAP,Snap Inc.,2020-07-21,TNS,-0.09,-0.09,1.10,EDT,0,EQUITY


Now that we have the dates... find the movement after earnings

In [16]:
historical_prices = yf.download(all_tickers, start='2020-07-03', end ='2020-09-14')['Adj Close']

[*********************100%***********************]  178 of 178 completed


In [17]:
# save to csv (so don't have to download every time)
historical_prices.to_csv('historical_prices.csv', index=True)

In [18]:
# read back in
historical_prices = pd.read_csv('historical_prices.csv', index_col=0)
historical_prices

,AAL,AAPL,ABBV,ABT,ALGN,AMAT,AMD,AMZN,APHA,ATVI,...,WIX,WMT,WORK,WYNN,ZG,ZM,ZNGA,ZS,ZTS,ZUMZ
Date,,,,,,,,,,,,,,,,,,,,,
2020-07-06,12.80,93.294289,97.816139,92.610466,282.440002,63.249836,53.400002,3057.040039,4.380,78.669998,...,276.559998,118.403229,31.120001,76.699997,60.980000,261.000000,9.72,114.779999,138.406052,27.959999
2020-07-07,11.91,93.004814,98.152031,92.102440,274.709991,61.974075,52.930000,3000.120117,4.330,78.500000,...,282.809998,126.430222,31.480000,73.370003,62.029999,262.100006,9.89,113.839996,138.246277,26.570000
2020-07-08,11.99,95.170906,98.082878,92.221977,278.390015,62.562122,53.430000,3081.110107,4.310,79.279999,...,292.119995,123.930504,34.240002,74.180000,63.200001,266.320007,10.25,120.849998,138.615753,26.690001
2020-07-09,11.18,95.580170,96.759041,93.307755,274.079987,63.279739,57.259998,3182.629883,4.190,80.639999,...,296.709991,127.226952,34.020000,71.129997,62.419998,269.519989,10.44,127.410004,137.487320,25.379999
2020-07-10,11.94,95.747368,95.662422,92.680199,273.290009,62.382717,55.880001,3200.000000,4.300,81.269997,...,294.230011,130.144943,33.840000,74.129997,61.750000,275.869995,10.38,125.489998,136.908142,25.750000
2020-07-13,11.63,95.305664,96.680000,92.729996,270.239990,61.306293,53.590000,3104.000000,4.680,77.949997,...,281.000000,128.989716,33.419998,81.269997,58.220001,260.299988,9.75,119.910004,138.116440,25.340000
2020-07-14,11.57,96.882820,98.870003,94.820000,299.619995,62.332886,54.720001,3084.000000,4.710,79.769997,...,279.820007,131.469513,32.369999,79.300003,57.750000,260.010010,9.82,122.430000,140.203522,25.549999
2020-07-15,13.44,97.549118,100.480003,96.730003,314.299988,61.894337,55.340000,3008.870117,5.070,79.150002,...,273.679993,131.459549,32.119999,86.739998,58.970001,256.220001,9.47,119.839996,142.720001,26.290001
2020-07-16,12.45,96.348778,99.919998,96.400002,309.000000,62.133545,54.919998,2999.899902,5.070,79.370003,...,271.420013,131.658737,31.610001,84.599998,61.740002,248.539993,9.43,117.150002,142.940002,26.309999


### Analysis/Data Visualization

#### What were the most "popular" stocks?

In [19]:
# what is the overall sentiment of each ticker?
overall_sentiment = {}
    
for ticker in all_tickers:
    mentioned = cleaned_comments['tickers'].apply(lambda x: ticker in x)
    senti = mentioned*cleaned_comments['sentiment_score']
    overall_sentiment[ticker] = sum(senti)

In [22]:
top50_sentiment = dict(heapq.nlargest(50, overall_sentiment.items(), key=lambda i: abs(i[1])))

#### What was each stock's earnings result?

In [56]:
earnings_surprise = {}

for ticker in top50_sentiment:
    earnings_surprise[ticker] = q2_earnings_calendar.loc[q2_earnings_calendar['ticker'] == ticker].reset_index()['epssurprisepct'][0]

#### How did each stock move?

Since we don't know from the data whether the earnings announcement was before market (stock movement same day) or after market (stock movement next day), we just calculate both and take the larger (in absolute value) movement.

In [58]:
# how did each stock price react to earnings?
holidays = [datetime(2020, 7, 3), datetime(2020, 9, 7)]
earnings_movement = {}

for ticker in top50_sentiment:
    # get earnings date, plus previous and next days.
    earn_date = q2_earnings_calendar.loc[q2_earnings_calendar['ticker'] == ticker].reset_index()['startdatetime'][0]
    prev_date = earn_date - CustomBusinessDay(1, holidays=holidays)
    next_date = earn_date + CustomBusinessDay(1, holidays=holidays)
    
    # change back to strings
    earn_date = str(earn_date)[:10]
    prev_date = str(prev_date)[:10]
    next_date = str(next_date)[:10]
    
    # calculate price movements
    prev_close = historical_prices.loc[prev_date][ticker]
    earn_close = historical_prices.loc[earn_date][ticker]
    next_close = historical_prices.loc[next_date][ticker]
    
    same_day_movement = (earn_close - prev_close)/prev_close
    next_day_movement = (next_close - earn_close)/earn_close
    
    if abs(same_day_movement) > abs(next_day_movement):
        earnings_movement[ticker] = same_day_movement
    else:
        earnings_movement[ticker] = next_day_movement

#### Aggregate Data and create visualizations

In [59]:
data = pd.DataFrame([top50_sentiment, earnings_surprise, earnings_movement],
                    index = ["WSB sentiment", "earnings surprise", "earnings movement"])

In [60]:
data.loc['earnings movement'] = data.loc['earnings movement'].apply(lambda x: 100*x) # change to percentage movement

In [61]:
# transpose for use with plotly
data = data.T
data['ticker'] = data.index

#### Sentiment bar chart

In [69]:
data = data.sort_values(by='WSB sentiment', ascending = False)

In [83]:
fig = px.bar(data, x='ticker', y='WSB sentiment', color='colours')
fig.update_layout(showlegend=False)
fig.show()

#### Compare with earnings surprise

Not shown: TWTR with huge negative earnings surprise

In [98]:
fig = px.scatter(data.loc[data['ticker'] != 'TWTR'], x='earnings surprise', y='WSB sentiment', text='ticker',
                labels = {'earnings surprise': 'earnings surprise (%)'})
fig.update_traces(textposition='top center', textfont_size=8)
fig.update_xaxes(zeroline=True, zerolinewidth=2, zerolinecolor='LightPink')
fig.update_yaxes(zeroline=True, zerolinewidth=2, zerolinecolor='LightPink')

fig.show()

In [116]:
stats.pearsonr(data['earnings surprise'], data['WSB sentiment'])

(0.29728659758448966, 0.03602677789245417)

#### Compare with stock movement

In [112]:
fig = px.scatter(data, x='earnings movement', y='WSB sentiment', text='ticker',
                labels = {'earnings movement': 'post-earnings movement (%)'})
fig.update_traces(textposition='top center', textfont_size=8)
fig.update_xaxes(zeroline=True, zerolinewidth=2, zerolinecolor='LightPink')
fig.update_yaxes(zeroline=True, zerolinewidth=2, zerolinecolor='LightPink')

fig.show()

In [117]:
stats.pearsonr(data['earnings movement'], data['WSB sentiment'])

(-0.03988336845232457, 0.7833205715691258)

#### Weighted portfolio

Let's say you made a weighted portfolio, weighted based upon the confidence of WSB investors. How would you have performed?

In [100]:
data['weight'] = data['WSB sentiment']/sum(data['WSB sentiment'])

In [102]:
portfolio_performance = data['earnings movement'].dot(data['weight'])
portfolio_performance

-1.6195161623652372